In [45]:
import requests
import pyrenn,pandas,copy
from pandas.io.json import json_normalize




def get_items_key(lis,key):
    return [k for k in lis if k["_T"]==key ]
    

   
def get_player_event(lis, pID,scores,idx):
    player_events=[]
    Y_hat=[]
    
    keys = ['LogItemEquip', 'LogItemUnequip', 'LogPlayerAttack', 'LogPlayerKill', 'LogPlayerMakeGroggy', \
                'LogPlayerTakeDamage', 'LogPlayerRevive']
    events=[event for event in lis]
    for event,event_score in zip(events,scores):
        if(event['_T'] in keys):
            for inkey in event.keys():
                if type(event[inkey]) is dict:
                    if('accountId' in event[inkey].keys()):
                        if event[inkey]["accountId"]==pID:
                            event_type=event["_T"]
                            timestamp=event["_D"]
                            
                            Y_hat.append(event_score[idx])
                            player_events.append({"type":event_type,"ID":pID,"_D":timestamp,event_type:event})
                            break
    return player_events,Y_hat
    
def get_outgoing_damage(row):
    if row["type"]=="LogPlayerTakeDamage":
        if "LogPlayerTakeDamage.attacker.accountId" in row.index:
            if row["ID"]==row["LogPlayerTakeDamage.attacker.accountId"]:
                return row["LogPlayerTakeDamage.damage"]
        else:
            return row["LogPlayerTakeDamage.damage"]
    return 0
    
def get_incoming_damage(row):
    if row["type"]=="LogPlayerTakeDamage":
        if row["ID"]==row["LogPlayerTakeDamage.victim.accountId"]:
            return row["LogPlayerTakeDamage.damage"]
    return 0
    

    
            
  
def get_health(row):
    if row["type"]=="LogItemEquip": return row["LogItemEquip.character.health"]
    if row["type"]=="LogItemUnequip": return row["LogItemUnequip.character.health"]
    if row["type"]=="LogPlayerAttack": return row["LogPlayerAttack.attacker.health"]
    if row["type"]=="LogPlayerKill":
        if row["ID"]==row["LogPlayerKill.victim.accountId"]:
            return row["LogPlayerKill.victim.health"]
        if row["ID"]==row["LogPlayerKill.killer.accountId"]:
            return row["LogPlayerKill.killer.health"]

        print(row)
        return 0
    if row["type"]=="LogPlayerMakeGroggy":
        if row["ID"]==row["LogPlayerMakeGroggy.victim.accountId"]:
            return row["LogPlayerMakeGroggy.victim.health"]
        if row["ID"]==row["LogPlayerMakeGroggy.attacker.accountId"]: ## TODO: change the attack ID to whate ver 495 is 
            return row["LogPlayerMakeGroggy.attacker.health"]

        print(row)
        return 0
    if row["type"]=="LogPlayerTakeDamage":
        if row["ID"]==row["LogPlayerTakeDamage.victim.accountId"]:
            return row["LogPlayerTakeDamage.victim.health"]
        if row["ID"]==row["LogPlayerTakeDamage.attacker.accountId"]:
            return row["LogPlayerTakeDamage.attacker.health"]

        print(row)
        return 0
                               
    
    
# {u'LogArmorDestroy',
#  u'LogItemDrop',
#  u'LogItemEquip',
#  u'LogItemPickup',
#  u'LogItemUnequip',
#  u'LogItemUse',   # {u'Item_Boost_EnergyDrink_C', u'Item_Boost_PainKiller_C', u'Item_Heal_Bandage_C', u'Item_Heal_FirstAid_C',u'Item_Heal_MedKit_C'},
#  # u'LogMatchDefinition',

#  # u'LogPlayerAttack',
#  u'LogPlayerCreate',
#  u'LogPlayerKill',
#  u'LogPlayerMakeGroggy',
#  u'LogPlayerPosition',
#  u'LogPlayerRevive',
#  u'LogPlayerTakeDamage',
#  u'LogVehicleDestroy',
#  u'LogVehicleLeave',
#  u'LogVehicleRide',
#  u'LogWheelDestroy'}
 
 
 

 # u'LogMatchEnd',
 # u'LogMatchStart',
 # for game in game_list:

    
    
    

In [ ]:
NeuralNetwork=    pyrenn.CreateNN([3,5,5,1],[0,1],[1],[1])
     
x=requests.get("https://telemetry-cdn.playbattlegrounds.com/bluehole-pubg/pc-tournament/2018/11/09/16/01/be5ef65e-e438-11e8-a2cb-0a58646eb6a9-telemetry.json")
y=x.json()

players = set([k["character"]["accountId"] for k in get_items_key(y,"LogPlayerCreate") if "BananaNPC" not in k["character"]["name"]])

latest_score = [0.0 for _ in players]
key_val_par={key:val for val,key in enumerate(players)}

scores_updated = []
scores_updated.append(latest_score)
already_killed = []
for event in y[1:]:
    if event['_T'] == 'LogPlayerKill':
        killer = event['killer']['accountId']
        killed = event['victim']['accountId']
        already_killed.append(killed)
        not_killed_keys = [key for key in key_val_par.keys() if key!= killed and key not in already_killed]
        for key in not_killed_keys:
            latest_score[key_val_par[key]] += 1
        try:latest_score[key_val_par[killed]] = 0.0
        except:pass
        
        scores_updated.append(copy.deepcopy(latest_score))
    else:
        scores_updated.append(copy.deepcopy(latest_score))

        

        
event_scores=[[0.0 for _ in players] for _ in y]        
        
        
for player in players:
    print(player)
    player_event_log_events,Y=get_player_event(y,player,scores_updated,key_val_par[player])
    player_event_log=json_normalize(player_event_log_events)
    P_frame=pandas.DataFrame([player_event_log.apply(get_health,axis=1).fillna(method="backfill"),
                              player_event_log.apply(get_outgoing_damage,axis=1).fillna(method="backfill"),
                              player_event_log.apply(get_incoming_damage,axis=1).fillna(method="backfill")]).values
    
    NeuralNetwork=pyrenn.train_LM(P_frame,pandas.DataFrame(Y).T.values,NeuralNetwork,verbose=True)
    
    data,network=pyrenn.prepare_data(P_frame,pandas.DataFrame(Y).T.values,NeuralNetwork)
    
    previous_index=0
    for event,prediction in zip(player_event_log_events,data["Y"]):
        event=event[[key for key in event.keys() if key not in ["type", "ID", "_D"]][0]]
        for i in range(previous_index,y.index(event)+1):
            event_scores[i][key_val_par[player]]=prediction
        previous_index=y.index(event)+1
    
    
    
print(pyrenn.calc_error(NeuralNetwork,data))

account.364286b1e16a47d6ae631e1ef5c838ad
('Iteration: ', 0, '\t\tError: ', 101.80547456924816, '\tscale factor: ', 3.0)
('Iteration: ', 1, '\t\tError: ', 90.99976509916127, '\tscale factor: ', 0.3)
('Iteration: ', 2, '\t\tError: ', 88.49125012727747, '\tscale factor: ', 3.0)
('Iteration: ', 3, '\t\tError: ', 87.5526333907595, '\tscale factor: ', 3.0)
('Iteration: ', 4, '\t\tError: ', 61.877899182863636, '\tscale factor: ', 3.0)
('Iteration: ', 5, '\t\tError: ', 57.72289383998545, '\tscale factor: ', 30.0)
('Iteration: ', 6, '\t\tError: ', 53.730246730873674, '\tscale factor: ', 30.0)
('Iteration: ', 7, '\t\tError: ', 52.77792650225719, '\tscale factor: ', 300.0)
('Iteration: ', 8, '\t\tError: ', 51.501353062373695, '\tscale factor: ', 300.0)
('Iteration: ', 9, '\t\tError: ', 49.17820005699029, '\tscale factor: ', 300.0)
('Iteration: ', 10, '\t\tError: ', 39.82312917948598, '\tscale factor: ', 300.0)
('Iteration: ', 11, '\t\tError: ', 25.09492266600856, '\tscale factor: ', 30.0)
('Itera

('Iteration: ', 100, '\t\tError: ', 6.1320313759887055, '\tscale factor: ', 30000.0)
Maximum number of iterations reached
account.e7e80dde2db14902b70e97831f360e92
('Iteration: ', 0, '\t\tError: ', 14.469837226691027, '\tscale factor: ', 3.0)
('Iteration: ', 1, '\t\tError: ', 2.8209723477258226, '\tscale factor: ', 0.3)
('Iteration: ', 2, '\t\tError: ', 2.1427526769349496, '\tscale factor: ', 0.3)
('Iteration: ', 3, '\t\tError: ', 2.046866995201885, '\tscale factor: ', 0.3)
('Iteration: ', 4, '\t\tError: ', 1.973944372608017, '\tscale factor: ', 0.3)
('Iteration: ', 5, '\t\tError: ', 1.8524355720680439, '\tscale factor: ', 0.3)
('Iteration: ', 6, '\t\tError: ', 1.2790638911136112, '\tscale factor: ', 0.03)
('Iteration: ', 7, '\t\tError: ', 0.9126088988674587, '\tscale factor: ', 0.03)
('Iteration: ', 8, '\t\tError: ', 0.39687647599643133, '\tscale factor: ', 0.03)
('Iteration: ', 9, '\t\tError: ', 0.28071018002623704, '\tscale factor: ', 0.003)
('Iteration: ', 10, '\t\tError: ', 0.07537

('Iteration: ', 88, '\t\tError: ', 0.0007542350330129843, '\tscale factor: ', 0.00030000000000000003)
('Iteration: ', 89, '\t\tError: ', 0.0007537159301639275, '\tscale factor: ', 0.00030000000000000003)
('Iteration: ', 90, '\t\tError: ', 0.0007532063316856012, '\tscale factor: ', 0.00030000000000000003)
('Iteration: ', 91, '\t\tError: ', 0.0007527059028278494, '\tscale factor: ', 0.00030000000000000003)
('Iteration: ', 92, '\t\tError: ', 0.0007522143359038039, '\tscale factor: ', 0.00030000000000000003)
('Iteration: ', 93, '\t\tError: ', 0.0007517313468529008, '\tscale factor: ', 0.00030000000000000003)
('Iteration: ', 94, '\t\tError: ', 0.0007512566722875902, '\tscale factor: ', 0.00030000000000000003)
('Iteration: ', 95, '\t\tError: ', 0.0007507900669497374, '\tscale factor: ', 0.00030000000000000003)
('Iteration: ', 96, '\t\tError: ', 0.0007503313015161271, '\tscale factor: ', 0.00030000000000000003)
('Iteration: ', 97, '\t\tError: ', 0.0007498801607030604, '\tscale factor: ', 0.00

('Iteration: ', 84, '\t\tError: ', 0.023519152924603128, '\tscale factor: ', 0.3)
('Iteration: ', 85, '\t\tError: ', 0.023396147585288907, '\tscale factor: ', 0.3)
('Iteration: ', 86, '\t\tError: ', 0.023278423890203653, '\tscale factor: ', 0.3)
('Iteration: ', 87, '\t\tError: ', 0.023165763693779696, '\tscale factor: ', 0.3)
('Iteration: ', 88, '\t\tError: ', 0.023057950023842932, '\tscale factor: ', 0.3)
('Iteration: ', 89, '\t\tError: ', 0.022954800115083367, '\tscale factor: ', 0.3)
('Iteration: ', 90, '\t\tError: ', 0.02285614491121755, '\tscale factor: ', 0.3)
('Iteration: ', 91, '\t\tError: ', 0.02276181787845034, '\tscale factor: ', 0.3)
('Iteration: ', 92, '\t\tError: ', 0.022671650713746258, '\tscale factor: ', 0.3)
('Iteration: ', 93, '\t\tError: ', 0.02258546827819002, '\tscale factor: ', 0.3)
('Iteration: ', 94, '\t\tError: ', 0.02250309536342441, '\tscale factor: ', 0.3)
('Iteration: ', 95, '\t\tError: ', 0.02242435295179759, '\tscale factor: ', 0.3)
('Iteration: ', 96, '

('Iteration: ', 80, '\t\tError: ', 4.5270523209412405e-05, '\tscale factor: ', 0.003)
('Iteration: ', 81, '\t\tError: ', 4.5055039880331196e-05, '\tscale factor: ', 0.003)
('Iteration: ', 82, '\t\tError: ', 4.4842267301167686e-05, '\tscale factor: ', 0.003)
('Iteration: ', 83, '\t\tError: ', 4.4632154583482085e-05, '\tscale factor: ', 0.003)
('Iteration: ', 84, '\t\tError: ', 4.442465214915607e-05, '\tscale factor: ', 0.003)
('Iteration: ', 85, '\t\tError: ', 4.42197116891134e-05, '\tscale factor: ', 0.003)
('Iteration: ', 86, '\t\tError: ', 4.401728611270679e-05, '\tscale factor: ', 0.003)
('Iteration: ', 87, '\t\tError: ', 4.381732951091936e-05, '\tscale factor: ', 0.003)
('Iteration: ', 88, '\t\tError: ', 4.361979711225592e-05, '\tscale factor: ', 0.003)
('Iteration: ', 89, '\t\tError: ', 4.342464525225879e-05, '\tscale factor: ', 0.003)
('Iteration: ', 90, '\t\tError: ', 4.3231831327819714e-05, '\tscale factor: ', 0.003)
('Iteration: ', 91, '\t\tError: ', 4.3041313772510534e-05, '\

In [48]:
NeuralNetwork.keys()

['layers',
 'L_b',
 'dL',
 'ErrorHistory',
 'nn',
 'dI',
 'I',
 'L_f',
 'M',
 'N',
 'delay',
 'normY',
 'U',
 'CU_LW',
 'w',
 'w0',
 'X',
 'dmax',
 'normP',
 'CX_LW']

In [21]:
pyrenn.saveNN(filename="NNET.pkl",net=NeuralNetwork)

In [47]:
data,network=pyrenn.prepare_data(P_frame,pandas.DataFrame(Y).T.values,NeuralNetwork)
print(pyrenn.calc_error(NeuralNetwork,data))

14.859364362838615


In [57]:
data["Y"]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.02380952, 0.02380952, 0.02380952, 0.04761905,
        0.04761905, 0.04761905, 0.04761905, 0.11904762, 0.11904762,
        0.14285714, 0.14285714, 0.14285714, 0.14285714, 0.23809524,
        0.23809524, 0.23809524, 0.23809524, 0.23809524, 0.23809524,
        0.23809524, 0.78571429, 0.78571429, 0.78571429, 0.78571429,
        0.78571429, 0.78571429, 0.78571429, 0.78571429, 0.78571429,
        0.78571429, 0.78571429, 0.78571429, 0.78571429, 0.78571429,
        0.78571429, 0.78571429, 0.78571429, 0.78571429, 0.78571429,
        0.78571429, 0.78571429, 0.78571429, 0.78571429, 0.78571429,
        0.88095238, 0.88095238, 0.88095238, 0.88095238, 0.88095238,
        0.88095238, 0.88095238, 0.88095238, 0.88095238, 0.88095238,
        0.88095238, 0.88095238, 0.88095238, 0.88095238, 0.88095238,
        0.88095238, 0.88095238, 0.88095238, 0.88095238, 0.88095238,
        0.88095238, 0.88095238, 0.88095238, 0.88

In [51]:
Y

[0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 5,
 5,
 6,
 6,
 6,
 6,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 33,
 33,
 33,
 33,
 33,
 33,
 33,
 33,
 33,
 33,
 33,
 33,
 33,
 33,
 33,
 33,
 33,
 33,
 33,
 33,
 33,
 33,
 33,
 33,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 39,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 41,
 41,
 41,
 41,
 41,
 41,
 41,
 41,
 41,
 41,
 41,
 41,
 41,
 41,
 41,
 41,
 42,
 42,
 42,
 42,
 43,
 43,
 43,
 43,
 44,
 44,
 44,
 44,
 44,
 44,
 44,
 44,
 44,
 44,
 44,
 44,
 44,
 44,
 44,
 44,
 44,
 44]

<Response [200]>

In [53]:
y.

[{u'MatchId': u'match.bro.custom.es_CN-PRSO2_04.steam.normal.as.2018.11.09.23.a2737609-b30d-45f4-a78c-41a463373967',
  u'PingQuality': u'',
  u'SeasonState': u'progress',
  u'_D': u'2018-11-09T15:35:23.7665987Z',
  u'_T': u'LogMatchDefinition'},
 {u'_D': u'2018-11-09T15:25:40.543Z',
  u'_T': u'LogPlayerLogin',
  u'accountId': u'account.498bc70dd3be4f1183c05ce71db6c55d',
  u'common': {u'isGame': 0}},
 {u'_D': u'2018-11-09T15:25:40.546Z',
  u'_T': u'LogPlayerLogin',
  u'accountId': u'account.06ac12b1890041d6bef9b2de314aef37',
  u'common': {u'isGame': 0}},
 {u'_D': u'2018-11-09T15:25:40.547Z',
  u'_T': u'LogPlayerLogin',
  u'accountId': u'account.49526f8af4454c55be97948fc6fd8c66',
  u'common': {u'isGame': 0}},
 {u'_D': u'2018-11-09T15:25:40.566Z',
  u'_T': u'LogItemEquip',
  u'character': {u'accountId': u'account.49526f8af4454c55be97948fc6fd8c66',
   u'health': 100,
   u'location': {u'x': 252129.078125,
    u'y': 137475.765625,
    u'z': 3987.777099609375},
   u'name': u'BananaNPC_43',
  

In [61]:
{str(k) for k in scores_updated}

{'[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]',
 '[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 44.0]',
 '[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 43.0, 43.0]',
 '[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]',
 '[10.0, 10.0, 10.0, 0.0, 10.0, 0.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 0.0, 0.0, 10.0, 0.0, 0.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 0.0, 10.0, 10.0, 10.0, 10.0]',
 '[11.0, 11.0, 11.0, 0.0, 11.0, 0.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 0.0, 0.0, 11.0, 0.0, 0.0, 0.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 0.0, 11.0, 11.0, 11.0, 11.0]',
 '[12.0, 12.0, 12.0, 0

In [81]:
min_max=[(k) for k in range(1,len(scores_updated)) if scores_updated[k-1]!=scores_updated[k]]
min_max.insert(0,0)
min_max=[[(min_max[i-1],min_max[i]) for i in range(1,len(min_max)) if min_max[i-1]<=j and min_max[i]>=j ]   for j in range(len(y))]       
[[  scores_updated[min_max[i][0][0]]*(min_max[i][0][1]-i)/(min_max[i][0][1]-min_max[i][0][0])+
  scores_updated[min_max[i][0][1]]*(i-min_max[i][0][0])/(min_max[i][0][1]-min_max[i][0][0])  
  for _ in players] for i in range(len(y))]

TypeError: unsupported operand type(s) for /: 'list' and 'int'

In [ ]:
import matplotlib as plt

In [ ]:
plt.plot(event_scores)